In [8]:
from pymongo import MongoClient
from pymongo import errors
from pprint import pprint
import numpy as np
import regex as re
import pandas as pd

In [2]:
def connect_database():
    client = MongoClient('localhost', 27017)
    return client

In [3]:
client = connect_database()

In [4]:
fragrances_collection = client.Perfume_Database.Fragrances

# Analyze the gathered data

## Companies collected in the database

The pipeline created, queries unique companies and aggregates the count of perfumes made by that company that are present in the collection. The results are sorted by perfume count in descending order.

In [6]:
extract_unique_companies_pipeline = [
    {
        '$group': {
            '_id': '$company', 
            'Perfumes_Count': {
                '$count': {}
            }
        }
    }, {
        '$project': {
            'Company': '$_id', 
            'Perfumes_Count': 1, 
            '_id': 0
        }
    }, {
        '$sort': {
            'Perfumes_Count': -1
        }
    }
]

In [7]:
unique_Brands = fragrances_collection.aggregate(extract_unique_companies_pipeline)

In [9]:
unique_Brands_df = pd.DataFrame(unique_Brands)

After importing the results of the aggregation, we can see the top 10 companies in regards to the number of perfumes made by them is present in the collection.

In [10]:
unique_Brands_df.head(10)

,Perfumes_Count,Company
0,51,montale
1,49,zara
2,47,xerjoff
3,44,serge lutens
4,42,byredo
5,42,l'artisan parfumeur
6,40,maison francis kurkdjian
7,39,tom ford private blend
8,39,amouage
9,38,profumum


In [12]:
unique_Brands_df.describe()

,Perfumes_Count
count,1065.000000
mean,4.276995
std,6.444040
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,51.000000


## Notes collected in the database

Extract unique notes

In [18]:
extract_count_notes = [
    {
        '$project': {
            'all_notes': {
                '$concatArrays': [
                    {
                        '$ifNull': [
                            '$top notes', []
                        ]
                    }, {
                        '$ifNull': [
                            '$base notes', []
                        ]
                    }, {
                        '$ifNull': [
                            '$middle notes', []
                        ]
                    }, {
                        '$ifNull': [
                            '$notes', []
                        ]
                    }
                ]
            }
        }
    }, {
        '$unwind': {
            'path': '$all_notes'
        }
    }, {
        '$group': {
            '_id': '$all_notes', 
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
]

In [19]:
unique_notes = fragrances_collection.aggregate(extract_count_notes)

In [20]:
unique_notes_df = pd.DataFrame(unique_notes)

In [21]:
unique_notes_df.head(20)

,_id,count
0,musk,1527
1,vanilla,1250
2,bergamot,1171
3,sandalwood,1162
4,patchouli,1154
5,amber,1134
6,jasmine,940
7,rose,846
8,cedar,749
9,vetiver,641


Number of notes in each perfume

Top most used notes overal

Top most used notes in each of Top, Base and middle notes